In [42]:
%pip install matplotlib seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 33.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [seaborn]m7/8 [seaborn]ib]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon
from shapely import wkt
from sklearn.cluster import KMeans, DBSCAN
from scipy.spatial import Voronoi
import numpy as np
from collections import defaultdict
import plotly.graph_objects as go
import geopandas as gpd
import folium
from folium import plugins
import plotly.graph_objects as go


In [45]:
parks = pd.read_csv("assets/parks.csv")
rest = pd.read_csv("assets/restaurants.csv")
metro = pd.read_csv("assets/metro.csv")
rest_info = pd.read_csv("assets/rest_info.csv")

print(f"The number of [parks]: {len(parks):>10}")
print(f"The number of [restaurants]: {len(rest):>4}")
print(f"The number of [metros]: {len(metro):>8}")
print(f"The number of [rest_info]: {len(rest_info):>8}")

The number of [parks]:       2055
The number of [restaurants]: 285290
The number of [metros]:      496
The number of [rest_info]:   291538


In [32]:
print(f"\n --- PARKS --- \n")
display(parks.head(n=5))

print(f"\n --- RESTAURANTS --- \n")
display(rest.head(n=5))

print(f"\n --- METRO --- \n")
display(metro.head(n=5))

print(f"\n --- REST_INFO --- \n")
display(rest_info.head(n=5))



 --- PARKS --- 



,ACQUISITIONDATE,ACRES,ADDRESS,BOROUGH,CLASS,COMMUNITYBOARD,COUNCILDISTRICT,DEPARTMENT,EAPPLY,GISOBJID,...,PIP_RATABLE,PRECINCT,RETIRED,SIGNNAME,SUBCATEGORY,TYPECATEGORY,US_CONGRESS,WATERFRONT,ZIPCODE,multipolygon
0,1972 Sep 22 12:00:00 AM,83.3,1000 RICHMOND TERRACE,R,PARK,501,49,R-01,Snug Harbor Cultural Center,"100,003,741",...,False,120,False,Snug Harbor Cultural Center,Large Park,Historic House Park,11,True,"10301, 10310",MULTIPOLYGON (((-74.10086903940999 40.64468646...
1,NaN,0.356,200-15 MCLAUGHLIN AV,Q,PARK,408,23,Q-08,McLaughlin Playground,"100,000,044",...,True,107,False,McLaughlin Playground,Neighborhood Plgd,Playground,5,False,11423,MULTIPOLYGON (((-73.76596231448613 40.72551186...
2,1926 Apr 29 12:00:00 AM,286.557,298 SATERLEE STREET,R,PARK,503,51,R-03,Conference House Park,"100,004,319",...,False,123,False,Conference House Park,Large Park,Nature Area,11,True,"10307, 10309",MULTIPOLYGON (((-74.25204073763335 40.50292259...
3,1990 Oct 26 12:00:00 AM,6.611,2 KENT STREET,B,PARK,301,33,B-01,WNYC Transmitter Park,"100,003,886",...,True,94,False,WNYC Transmitter Park,Neighborhood Park,Neighborhood Park,NaN,True,11222,MULTIPOLYGON (((-73.96140516032709 40.73032004...
4,1948 Jan 29 12:00:00 AM,1.534,255 PARKINSON AVENUE,R,PARK,502,50,R-02B,Old Town Playground,"100,004,659",...,True,122,False,Old Town Playground,JOP,Playground,11,False,10305,MULTIPOLYGON (((-74.08082911468377 40.59514394...



 --- RESTAURANTS --- 



,CAMIS,INSPECTION DATE,VIOLATION CODE
0,40375108,11/26/2025,04H
1,40375108,11/26/2025,10G
2,40395865,11/26/2025,08A
3,40395865,11/26/2025,10H
4,40395865,11/26/2025,04M



 --- METRO --- 



,GTFS Stop ID,Station ID,Complex ID,Division,Line,Stop Name,Borough,CBD,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Northbound,ADA Southbound,ADA Notes,Georeference
0,R01,1,1,BMT,Astoria,Astoria-Ditmars Blvd,Q,False,N W,Elevated,40.775036,-73.912034,Last Stop,Manhattan,0,0,0,NaN,POINT (-73.912034 40.775036)
1,R03,2,2,BMT,Astoria,Astoria Blvd,Q,False,N W,Elevated,40.770258,-73.917843,Astoria,Manhattan,1,1,1,NaN,POINT (-73.917843 40.770258)
2,R04,3,3,BMT,Astoria,30 Av,Q,False,N W,Elevated,40.766779,-73.921479,Astoria,Manhattan,0,0,0,NaN,POINT (-73.921479 40.766779)
3,R05,4,4,BMT,Astoria,Broadway,Q,False,N W,Elevated,40.761820,-73.925508,Astoria,Manhattan,0,0,0,NaN,POINT (-73.925508 40.76182)
4,R06,5,5,BMT,Astoria,36 Av,Q,False,N W,Elevated,40.756804,-73.929575,Astoria,Manhattan,0,0,0,NaN,POINT (-73.929575 40.756804)



 --- REST_INFO --- 



,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location
0,50168829,Z-GRILL & PIZZERIA,Manhattan,803,8 AVENUE,10019.0,2123151659,NaN,01/01/1900,NaN,...,NaN,40.761364,-73.986785,104.0,3.0,12700.0,1025144.0,1.010390e+09,MN15,POINT (-73.986784542496 40.761364409897)
1,50159261,FROZEN LOVE INC,Manhattan,122,LA SALLE STREET,10027.0,9173256400,NaN,01/01/1900,NaN,...,NaN,40.813967,-73.960185,109.0,7.0,21100.0,1059842.0,1.019930e+09,MN09,POINT (-73.960184888403 40.813966918793)
2,50063280,ENZO'S OF ARTHUR AVE,Bronx,NaN,ARTHUR AVENUE,NaN,7187334455,Italian,05/28/2025,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,50179242,OLIO E PIU,Manhattan,5,BRYANT PARK,10018.0,2122047111,NaN,01/01/1900,NaN,...,NaN,40.753937,-73.985949,105.0,4.0,11300.0,1022565.0,1.009930e+09,MN17,POINT (-73.985948643127 40.753936992473)
4,50168250,NEWLAB,Brooklyn,63,FLUSHING AVENUE,11205.0,8083992969,NaN,01/01/1900,NaN,...,NaN,40.698013,-73.974986,302.0,33.0,54300.0,3335129.0,3.020230e+09,BK99,POINT (-73.974986348464 40.698013257962)


In [46]:


NYC_METRIC_CRS = "EPSG:2263"


class EventHotspotDistributor:
    """
    A class to find optimal event locations and distribute people equally
    across parks, metro stations, and restaurants in NYC.
    """

    def __init__(self, parks_df, metro_df, rest_df, rest_info_df):
        """
        Initialize with four datasets:
        - parks_df: CSV with 'multipolygon' column
        - metro_df: CSV with 'Georeference' (POINT WKT)
        - rest_df: inspection data (CAMIS-based)
        - rest_info_df: restaurant metadata with Latitude/Longitude
        """
        self.parks_df = parks_df.copy()
        self.metro_df = metro_df.copy()
        self.rest_df = rest_df.copy()
        self.rest_info_df = rest_info_df.copy()


        self.parks = self._parse_parks(parks_df)
        self.metro = self._parse_metro(metro_df)
        self.restaurants = self._get_restaurant_points()


        self.parks_metric = self.parks.to_crs(NYC_METRIC_CRS)
        self.metro_metric = self.metro.to_crs(NYC_METRIC_CRS)
        self.restaurants_metric = self.restaurants.to_crs(NYC_METRIC_CRS)


        self.parks_wgs84 = self.parks_metric.to_crs("EPSG:4326")
        self.metro_wgs84 = self.metro_metric.to_crs("EPSG:4326")
        self.restaurants_wgs84 = self.restaurants_metric.to_crs("EPSG:4326")

    def _parse_parks(self, df):
        """Parsing parks CSV with multipolygon column"""
        df = df.copy()
        df["geometry"] = df["multipolygon"].apply(wkt.loads)
        return gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")


    def _parse_metro(self, df):
        """Parsing metro CSV with Georeference column (POINT format)"""
        df = df.copy()
        df["geometry"] = df["Georeference"].apply(wkt.loads)
        return gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

    def _get_restaurant_points(self):
        """
        Extracting unique restaurant locations from rest_info.csv
        Uses Latitude/Longitude columns, removes duplicates by CAMIS
        """
        df = self.rest_info_df.copy()
        df = df.dropna(subset=["Latitude", "Longitude"])
        df = df.drop_duplicates(subset=["CAMIS"])

        geometry = [Point(xy) for xy in zip(df["Longitude"], df["Latitude"])]
        gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
        return gdf[gdf.geometry.notnull()]

    def create_weighted_points(self, weight_parks=4, weight_metro=15, weight_restaurant=3):
        """
        Combining all datasets with weights for importance
        Metro has the highest weight → hotspots near transport hubs
        """
        points = []


        for _, row in self.parks_wgs84.iterrows():
            c = row.geometry.centroid
            points.extend([[c.x, c.y]] * weight_parks)


        for _, row in self.metro_wgs84.iterrows():
            points.extend([[row.geometry.x, row.geometry.y]] * weight_metro)


        for _, row in self.restaurants_wgs84.iterrows():
            points.extend([[row.geometry.x, row.geometry.y]] * weight_restaurant)

        return np.array(points)

    def find_hotspots(self, n_hotspots=12, method='kmeans'):
        """
        Finding optimal hotspot locations using weighted KMeans
        """
        coords = self.create_weighted_points(weight_parks=4, weight_metro=20, weight_restaurant=3)

        kmeans = KMeans(n_clusters=n_hotspots, random_state=42, n_init=25)
        kmeans.fit(coords)

        centers = gpd.GeoDataFrame(
            {"hotspot_id": range(n_hotspots)},
            geometry=[Point(lon, lat) for lon, lat in kmeans.cluster_centers_],
            crs="EPSG:4326"
        ).to_crs(NYC_METRIC_CRS)


        radius_m = 800
        buffer = centers.geometry.buffer(radius_m)
        centers["parks_near"] = [self.parks_metric.intersects(b).sum() for b in buffer]
        centers["metro_near"] = [self.metro_metric.intersects(b).sum() for b in buffer]
        centers["rest_near"] = [self.restaurants_metric.intersects(b).sum() for b in buffer]

        centers["score"] = (
            centers["metro_near"] * 6 +
            centers["rest_near"] * 2 +
            centers["parks_near"] * 1.5
        )

        return centers.sort_values("score", ascending=False).reset_index(drop=True).to_crs("EPSG:4326")

    def distribute_people_equally(self, hotspots, total_people=18000):
        """
        Distributing people equally across hotspots using capacity constraints
        Better locations get +1 person bonus
        """
        n = len(hotspots)
        base = total_people // n
        extra = total_people % n

        h = hotspots.copy().sort_values("score", ascending=False).reset_index(drop=True)
        h["people"] = [base + (1 if i < extra else 0) for i in range(n)]
        return h

    def create_service_areas(self, hotspots):
        """
        Creating service areas (Voronoi or simple buffer) for each hotspot
        """
        hotspots_metric = hotspots.to_crs(NYC_METRIC_CRS)
        coords = np.array([[p.x, p.y] for p in hotspots_metric.geometry])
        vor = Voronoi(coords)


        service_areas = hotspots_metric.copy()
        service_areas["geometry"] = service_areas.geometry.buffer(1500)  # ~1.5 км
        return service_areas.to_crs("EPSG:4326")

    def assign_venues_to_hotspots(self, hotspots):
        """
        Assigning parks, metro stations, and restaurants to nearest hotspot
        """
        hotspots_metric = hotspots.to_crs(NYC_METRIC_CRS)
        assignment = defaultdict(lambda: {"parks": 0, "metro": 0, "rest": 0})

        for _, p in self.parks_metric.iterrows():
            nearest = hotspots_metric.geometry.distance(p.geometry.centroid).idxmin()
            assignment[nearest]["parks"] += 1

        for _, m in self.metro_metric.iterrows():
            nearest = hotspots_metric.geometry.distance(m.geometry).idxmin()
            assignment[nearest]["metro"] += 1

        for _, r in self.restaurants_metric.iterrows():
            nearest = hotspots_metric.geometry.distance(r.geometry).idxmin()
            assignment[nearest]["rest"] += 1

        return dict(assignment)

    def create_hexagonal_grid(self, resolution_km=1.0):
        """
        Creating equal-area hexagonal grid (placeholder — can be replaced with H3)
        """
        bounds = self.parks_metric.total_bounds

        cell_size = resolution_km * 900  # грубо в метрах
        x_coords = np.arange(bounds[0], bounds[2], cell_size)
        y_coords = np.arange(bounds[1], bounds[3], cell_size)

        cells = []
        for x in x_coords:
            for y in y_coords:
                cell = Point(x, y).buffer(cell_size / 2)
                if self.parks_metric.intersects(cell).any():
                    cells.append(cell)

        grid = gpd.GeoDataFrame({"geometry": cells}, crs=NYC_METRIC_CRS)
        return grid.to_crs("EPSG:4326")

    def get_summary_stats(self):
        """Generate comprehensive summary statistics of the NYC datasets"""
        # Basic counts
        total_parks = len(self.parks)
        total_metro = len(self.metro)
        unique_restaurants = len(self.restaurants)
        total_inspections = len(self.rest_df)

        # Borough distribution
        borough_stats = self.parks_df["BOROUGH"].value_counts()
        borough_mapping = {'B': 'Bronx', 'M': 'Manhattan', 'Q': 'Queens', 'R': 'Staten Island', 'X': 'Brooklyn'}

        # Restaurant grades distribution
        grade_counts = self.rest_df['GRADE'].value_counts() if 'GRADE' in self.rest_df.columns else pd.Series()

        # Geographic coverage
        total_area = self.parks_metric.geometry.area.sum() / 1e6  # Convert to km²

        # Format the output
        output = []
        output.append("=" * 60)
        output.append("NYC CITY PULSE LAB - DATASET SUMMARY")
        output.append("=" * 60)

        output.append("\nGEOGRAPHIC VENUES:")
        output.append(f"   • Parks: {total_parks:,} total ({total_area:.1f} km² area)")
        output.append(f"   • Metro Stations: {total_metro:,}")
        output.append(f"   • Restaurants: {unique_restaurants:,} unique locations")

        output.append("\nPARKS BY BOROUGH:")
        for code, count in borough_stats.items():
            borough_name = borough_mapping.get(code, code)
            percentage = (count / total_parks) * 100
            output.append(f"   • {borough_name}: {count} parks ({percentage:.1f}%)")

        output.append("\nRESTAURANT INSPECTIONS:")
        output.append(f"   • Total Inspections: {total_inspections:,}")
        output.append(f"   • Average Inspections per Restaurant: {total_inspections / unique_restaurants:.1f}")

        if grade_counts.any():
            output.append("\nHEALTH GRADES DISTRIBUTION:")
            total_graded = grade_counts.sum()
            for grade, count in grade_counts.items():
                percentage = (count / total_graded) * 100
                output.append(f"   • Grade {grade}: {count:,} ({percentage:.1f}%)")

        output.append("\nANALYSIS READY:")
        output.append("   • Coordinate Systems: EPSG:4326 (WGS84) + EPSG:2263 (NYC Metric)")
        output.append("   • Geometric Processing: ✓ Complete")
        output.append("   • Data Validation: ✓ Passed")

        output.append("=" * 60)

        return "\n".join(output)

distributor = EventHotspotDistributor(parks, metro, rest, rest_info)

print(distributor.get_summary_stats())

hotspots = distributor.find_hotspots(n_hotspots=12)
hotspots = distributor.distribute_people_equally(hotspots, total_people=20000)

display(hotspots[["hotspot_id", "metro_near", "rest_near", "parks_near", "score", "people"]].round(0))

NYC CITY PULSE LAB - DATASET SUMMARY

GEOGRAPHIC VENUES:
   • Parks: 2,055 total (1261.7 km² area)
   • Metro Stations: 496
   • Restaurants: 30,233 unique locations

PARKS BY BOROUGH:
   • Bronx: 626 parks (30.5%)
   • Queens: 475 parks (23.1%)
   • Brooklyn: 398 parks (19.4%)
   • Manhattan: 395 parks (19.2%)
   • Staten Island: 161 parks (7.8%)

RESTAURANT INSPECTIONS:
   • Total Inspections: 285,290
   • Average Inspections per Restaurant: 9.4

ANALYSIS READY:
   • Coordinate Systems: EPSG:4326 (WGS84) + EPSG:2263 (NYC Metric)
   • Geometric Processing: ✓ Complete
   • Data Validation: ✓ Passed


,hotspot_id,metro_near,rest_near,parks_near,score,people
0,1,0,359,0,718.0,1667
1,11,1,131,0,268.0,1667
2,0,1,114,4,240.0,1667
3,5,1,5,6,25.0,1667
4,4,0,6,0,12.0,1667
5,9,0,6,0,12.0,1667
6,6,0,4,2,11.0,1667
7,8,0,5,0,10.0,1667
8,7,0,4,0,8.0,1666
9,3,0,1,2,5.0,1666


In [ ]:
import geopandas as gpd
import pandas as pd
import folium
from folium import plugins
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
from shapely import wkt
import numpy as np
from branca.colormap import LinearColormap

class NYCMapVisualizer:
    """
    Create stunning interactive maps of NYC with parks, metro, and hotspots
    """

    def _get_event_suggestions(self, capacity):
        """Suggest event types based on capacity"""
        if capacity < 500:
            return "Small gatherings, picnics, workshops, yoga classes"
        elif capacity < 2000:
            return "Weddings, corporate events, festivals, concerts"
        elif capacity < 5000:
            return "Large festivals, concerts, sports events, conferences"
        else:
            return "Major festivals, concerts, marathons, mega events"

    def calculate_venue_capacity(self, park_row):
        """
        Estimate venue capacity based on park size and type
        """
        acres = park_row.get('ACRES', 0)
        if pd.isna(acres):
            acres = 0

        park_type = str(park_row.get('TYPECATEGORY', '')).lower()

        # Base capacity: ~100 people per acre for outdoor events
        base_capacity = acres * 100

        # Adjust based on park type
        if 'playground' in park_type:
            capacity = min(base_capacity, 500)  # Smaller venues
        elif 'large park' in park_type or 'flagship' in park_type:
            capacity = base_capacity * 1.5  # Can hold more
        elif 'triangle' in park_type or 'mall' in park_type:
            capacity = min(base_capacity, 300)  # Limited space
        else:
            capacity = base_capacity

        return int(max(capacity, 50))  # Minimum 50 people

    def categorize_venue_size(self, capacity):
        """Categorize venue by capacity"""
        if capacity < 500:
            return 'small', '🟢'  # Small gatherings
        elif capacity < 2000:
            return 'medium', '🟡'  # Medium events
        elif capacity < 5000:
            return 'large', '🟠'  # Large events
        else:
            return 'xlarge', '🔴'  # Mega events

    def __init__(self, parks_df, metro_df, rest_df=None):
        """Initialize with NYC data"""
        self.parks = self._parse_parks(parks_df)
        self.metro = self._parse_metro(metro_df)
        self.rest_df = rest_df

        # Calculate capacities for all parks immediately
        self.parks['capacity'] = self.parks.apply(self.calculate_venue_capacity, axis=1)
        self.parks['size_category'] = self.parks['capacity'].apply(
            lambda x: self.categorize_venue_size(x)[0]
        )

        # NYC center coordinates
        self.nyc_center = [40.7128, -74.0060]

        # Subway line colors (MTA official colors)
        self.subway_colors = {
            'A': '#0039A6', 'C': '#0039A6', 'E': '#0039A6',  # Blue
            'B': '#FF6319', 'D': '#FF6319', 'F': '#FF6319', 'M': '#FF6319',  # Orange
            'G': '#6CBE45',  # Light Green
            'J': '#996633', 'Z': '#996633',  # Brown
            'L': '#A7A9AC',  # Gray
            'N': '#FCCC0A', 'Q': '#FCCC0A', 'R': '#FCCC0A', 'W': '#FCCC0A',  # Yellow
            'S': '#808183',  # Gray Shuttle
            '1': '#EE352E', '2': '#EE352E', '3': '#EE352E',  # Red
            '4': '#00933C', '5': '#00933C', '6': '#00933C',  # Green
            '7': '#B933AD',  # Purple
        }

    def _parse_parks(self, df):
        """Parse parks from CSV"""
        df['geometry'] = df['multipolygon'].apply(wkt.loads)
        # Convert ACRES to numeric, handling any non-numeric values
        df['ACRES'] = pd.to_numeric(df['ACRES'], errors='coerce')
        return gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

    def _parse_metro(self, df):
        """Parse metro from CSV"""
        df['geometry'] = df['Georeference'].apply(wkt.loads)
        return gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

    def create_folium_masterpiece(self, hotspots=None, save_path='nyc_map.html'):
        """
        Create an EPIC Folium map with all the bells and whistles
        """
        # Create base map with dark theme
        m = folium.Map(
            location=self.nyc_center,
            zoom_start=11,
            tiles='CartoDB dark_matter',
            prefer_canvas=True,
            attr='CartoDB'
        )

        # Add alternative tile layers with proper attribution
        folium.TileLayer(
            tiles='CartoDB positron',
            name='CartoDB Positron',
            attr='CartoDB'
        ).add_to(m)

        folium.TileLayer(
            tiles='OpenStreetMap',
            name='OpenStreetMap',
            attr='OpenStreetMap contributors'
        ).add_to(m)

        # Parks Layer - SEPARATED BY SIZE for filtering
        small_parks = folium.FeatureGroup(name='🟢 Small Venues (<500 people)', show=True)
        medium_parks = folium.FeatureGroup(name='🟡 Medium Venues (500-2K)', show=True)
        large_parks = folium.FeatureGroup(name='🟠 Large Venues (2K-5K)', show=True)
        xlarge_parks = folium.FeatureGroup(name='🔴 Mega Venues (5K+)', show=True)

        # Map size categories to feature groups
        size_groups = {
            'small': small_parks,
            'medium': medium_parks,
            'large': large_parks,
            'xlarge': xlarge_parks
        }

        # Color schemes for each size
        size_colors = {
            'small': {'fill': '#90EE90', 'border': '#32CD32'},
            'medium': {'fill': '#FFD700', 'border': '#FFA500'},
            'large': {'fill': '#FF8C00', 'border': '#FF4500'},
            'xlarge': {'fill': '#FF0000', 'border': '#8B0000'}
        }

        for idx, park in self.parks.iterrows():
            acres = park.get('ACRES', 0)
            if pd.isna(acres):
                acres = 0

            capacity = park['capacity']
            size_cat = park['size_category']
            size_emoji = self.categorize_venue_size(capacity)[1]
            colors = size_colors[size_cat]

            # Get centroid using projected coordinates to avoid warning
            park_geom = gpd.GeoSeries([park.geometry], crs=self.parks.crs)
            centroid = park_geom.to_crs('EPSG:2263').centroid.to_crs('EPSG:4326').iloc[0]

            # Create enhanced popup with capacity info
            popup_html = f"""
            <div style="font-family: Arial; width: 300px;">
                <h3 style="color: {colors['border']}; margin: 0; border-bottom: 3px solid {colors['border']};">
                    {size_emoji} {park.get('SIGNNAME', 'Unknown Park')}
                </h3>
                <div style="padding: 10px 0;">
                    <p style="margin: 5px 0; font-size: 16px; font-weight: bold; color: {colors['border']};">
                        👥 Capacity: ~{capacity:,} people
                    </p>
                    <hr style="margin: 8px 0;">
                    <b>🏞️ Size:</b> {acres:.2f} acres<br>
                    <b>📍 Borough:</b> {park.get('BOROUGH', 'N/A')}<br>
                    <b>📮 Zip:</b> {park.get('ZIPCODE', 'N/A')}<br>
                    <b>🏛️ Type:</b> {park.get('TYPECATEGORY', 'N/A')}<br>
                    <br>
                    <div style="background: {colors['fill']}; padding: 8px; border-radius: 5px; text-align: center;">
                        <b>Perfect for:</b><br>
                        {self._get_event_suggestions(capacity)}
                    </div>
                </div>
            </div>
            """

            # Add to appropriate size group
            feature_group = size_groups[size_cat]

            folium.GeoJson(
                park.geometry,
                style_function=lambda x, fc=colors['fill'], bc=colors['border']: {
                    'fillColor': fc,
                    'color': bc,
                    'weight': 2,
                    'fillOpacity': 0.6
                },
                popup=folium.Popup(popup_html, max_width=350)
            ).add_to(feature_group)

        # Add all size groups to map
        small_parks.add_to(m)
        medium_parks.add_to(m)
        large_parks.add_to(m)
        xlarge_parks.add_to(m)

        # Metro Lines Layer - AWESOME subway lines
        metro_group = folium.FeatureGroup(name='🚇 Subway Stations', show=True)

        # Group stations by line for better visualization
        for idx, station in self.metro.iterrows():
            lines = str(station.get('Daytime Routes', '')).strip()
            if not lines:
                continue

            # Get color for first line
            first_line = lines.split()[0] if lines else 'S'
            color = self.subway_colors.get(first_line, '#808183')

            # Create awesome popup
            ada_status = '✅ Accessible' if station.get('ADA', 0) == 1 else '❌ Not Accessible'

            popup_html = f"""
            <div style="font-family: Arial; width: 280px; background: linear-gradient(135deg, {color}22, white);">
                <h3 style="color: {color}; margin: 0; padding: 10px; background: white; border-left: 5px solid {color};">
                    🚇 {station.get('Stop Name', 'Unknown')}
                </h3>
                <div style="padding: 10px;">
                    <b>🚊 Lines:</b> <span style="font-size: 18px; font-weight: bold; color: {color};">{lines}</span><br>
                    <b>📍 Borough:</b> {station.get('Borough', 'N/A')}<br>
                    <b>♿ ADA:</b> {ada_status}<br>
                    <b>🏗️ Structure:</b> {station.get('Structure', 'N/A')}
                </div>
            </div>
            """

            # Circle marker with subway line color
            folium.CircleMarker(
                location=[station.geometry.y, station.geometry.x],
                radius=8,
                popup=folium.Popup(popup_html, max_width=300),
                color=color,
                fill=True,
                fillColor=color,
                fillOpacity=0.8,
                weight=3
            ).add_to(metro_group)

        metro_group.add_to(m)

        # Hotspots Layer (if provided)
        if hotspots is not None:
            hotspots_group = folium.FeatureGroup(name='🔥 Event hotspots', show=True)

            for idx, hotspot in hotspots.iterrows():
                # Pulsing marker for hotspots
                folium.CircleMarker(
                    location=[hotspot.geometry.y, hotspot.geometry.x],
                    radius=15,
                    popup=f"""
                    <div style="font-family: Arial;">
                        <h3>🔥 Hotspot #{hotspot['hotspot_id']}</h3>
                        <b>🌳 Nearby parks:</b> {hotspot.get('nearby_parks', 0)}<br>
                        <b>🚇 Nearby Metro:</b> {hotspot.get('nearby_metro', 0)}<br>
                        <b>⭐ Score:</b> {hotspot.get('nearby_metro', 0) * 2 + hotspot.get('nearby_parks', 0)}
                    </div>
                    """,
                    color='#FF4500',
                    fill=True,
                    fillColor='#FFD700',
                    fillOpacity=0.7,
                    weight=4
                ).add_to(hotspots_group)

            hotspots_group.add_to(m)

        # Add heatmap of metro density
        heat_data = [[station.geometry.y, station.geometry.x] for _, station in self.metro.iterrows()]
        plugins.HeatMap(heat_data, name='🔥 Metro Density', radius=15, blur=25, max_zoom=13).add_to(m)

        # Add marker cluster for parks (optional, hidden by default)
        marker_cluster = plugins.MarkerCluster(name='📍 Park Clusters', show=False)

        # Use projected CRS for centroids
        parks_projected = self.parks.to_crs('EPSG:2263')
        park_centroids = parks_projected.geometry.centroid.to_crs('EPSG:4326')

        for idx, (park, centroid) in enumerate(zip(self.parks.itertuples(), park_centroids)):
            folium.Marker(
                location=[centroid.y, centroid.x],
                popup=park.SIGNNAME if hasattr(park, 'SIGNNAME') else 'Park',
                icon=folium.Icon(color='green', icon='tree', prefix='fa')
            ).add_to(marker_cluster)
        marker_cluster.add_to(m)

        # Add fullscreen button
        plugins.Fullscreen().add_to(m)

        # Add measure tool
        plugins.MeasureControl().add_to(m)

        # Add minimap
        plugins.MiniMap().add_to(m)

        # Add layer control
        folium.LayerControl(collapsed=False).add_to(m)

        # Add custom legend with capacity guide
        legend_html = '''
        <div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000;
                    background-color: white; padding: 20px; border: 3px solid #333;
                    border-radius: 15px; font-family: Arial; box-shadow: 0 0 20px rgba(0,0,0,0.3);
                    max-width: 350px;">
            <h3 style="margin: 0 0 15px 0; color: #333; border-bottom: 2px solid #333; padding-bottom: 10px;">
                🗽 NYC Event Venue Finder
            </h3>

            <div style="margin: 10px 0;">
                <h4 style="margin: 10px 0 5px 0; color: #666;">Venue Capacity Guide:</h4>
                <p style="margin: 5px 0; padding: 8px; background: #90EE90; border-radius: 5px;">
                    <b>🟢 Small (50-500)</b><br>
                    <span style="font-size: 12px;">Gatherings, workshops, picnics</span>
                </p>
                <p style="margin: 5px 0; padding: 8px; background: #FFD700; border-radius: 5px;">
                    <b>🟡 Medium (500-2K)</b><br>
                    <span style="font-size: 12px;">Weddings, corporate events</span>
                </p>
                <p style="margin: 5px 0; padding: 8px; background: #FF8C00; border-radius: 5px;">
                    <b>🟠 Large (2K-5K)</b><br>
                    <span style="font-size: 12px;">Festivals, concerts, sports</span>
                </p>
                <p style="margin: 5px 0; padding: 8px; background: #FF6347; border-radius: 5px;">
                    <b>🔴 Mega (5K+)</b><br>
                    <span style="font-size: 12px;">Major festivals, marathons</span>
                </p>
            </div>

            <hr style="margin: 10px 0;">

            <div style="margin: 10px 0;">
                <p style="margin: 3px 0; font-size: 13px;">
                    <span style="color: #0039A6;">🚇</span> <b>Subway Stations</b>
                </p>
                <p style="margin: 3px 0; font-size: 13px;">
                    <span style="color: #FF4500;">🔥</span> <b>Event Hotspots</b>
                </p>
            </div>

            <div style="margin-top: 15px; padding: 10px; background: #f0f0f0; border-radius: 5px;">
                <p style="margin: 0; font-size: 11px; color: #666;">
                    💡 <b>Tip:</b> Use the layer control (top right) to filter venues by size!
                </p>
            </div>
        </div>
        '''
        m.get_root().html.add_child(folium.Element(legend_html))

        # Save map
        m.save(save_path)
        print(f"✨ Epic Folium map saved to {save_path}")
        return m

    def create_plotly_3d_masterpiece(self, hotspots=None, save_path='nyc_3d_map.html'):
        """
        Create a stunning 3D Plotly visualization
        """
        fig = go.Figure()

        # Add parks as 3D scatter (height = acres)
        # Use projected CRS for accurate centroids
        parks_projected = self.parks.to_crs('EPSG:2263')
        park_centroids = parks_projected.geometry.centroid.to_crs('EPSG:4326')

        park_lons = [c.x for c in park_centroids]
        park_lats = [c.y for c in park_centroids]
        park_acres = self.parks['ACRES'].fillna(0).values
        park_names = self.parks['SIGNNAME'].fillna('Unknown').values

        fig.add_trace(go.Scatter3d(
            x=park_lons,
            y=park_lats,
            z=park_acres,
            mode='markers',
            marker=dict(
                size=8,
                color=park_acres,
                colorscale='Greens',
                showscale=True,
                colorbar=dict(title="Park Size (acres)"),
                line=dict(color='darkgreen', width=1)
            ),
            text=[f"🌳 {name}<br>Size: {acres:.1f} acres"
                  for name, acres in zip(park_names, park_acres)],
            hovertemplate='<b>%{text}</b><extra></extra>',
            name='Parks'
        ))

        # Add metro stations
        metro_lons = [station.geometry.x for _, station in self.metro.iterrows()]
        metro_lats = [station.geometry.y for _, station in self.metro.iterrows()]
        metro_names = self.metro['Stop Name'].fillna('Unknown').values
        metro_lines = self.metro['Daytime Routes'].fillna('').values

        fig.add_trace(go.Scatter3d(
            x=metro_lons,
            y=metro_lats,
            z=[5] * len(metro_lons),  # Fixed height
            mode='markers',
            marker=dict(
                size=6,
                color='red',
                symbol='diamond',
                line=dict(color='darkred', width=2)
            ),
            text=[f"🚇 {name}<br>Lines: {lines}"
                  for name, lines in zip(metro_names, metro_lines)],
            hovertemplate='<b>%{text}</b><extra></extra>',
            name='Metro Stations'
        ))

        # Add hotspots if provided
        if hotspots is not None:
            hotspot_lons = [h.geometry.x for _, h in hotspots.iterrows()]
            hotspot_lats = [h.geometry.y for _, h in hotspots.iterrows()]

            fig.add_trace(go.Scatter3d(
                x=hotspot_lons,
                y=hotspot_lats,
                z=[20] * len(hotspot_lons),
                mode='markers',
                marker=dict(
                    size=15,
                    color='gold',
                    symbol='diamond',
                    line=dict(color='orange', width=3)
                ),
                text=[f"🔥 Hotspot {i}" for i in range(len(hotspot_lons))],
                hovertemplate='<b>%{text}</b><extra></extra>',
                name='Hotspots'
            ))

        # Update layout for epic 3D view
        fig.update_layout(
            title={
                'text': '🗽 NYC Interactive 3D Map: Parks, Metro & Hotspots',
                'font': {'size': 24, 'color': '#1f77b4'}
            },
            scene=dict(
                xaxis_title='Longitude',
                yaxis_title='Latitude',
                zaxis_title='Height (Park Acres)',
                bgcolor='#f0f0f0',
                camera=dict(
                    eye=dict(x=1.5, y=1.5, z=1.3)
                )
            ),
            height=800,
            showlegend=True,
            legend=dict(
                x=0.02,
                y=0.98,
                bgcolor='rgba(255,255,255,0.8)'
            )
        )

        fig.write_html(save_path)
        print(f"✨ Epic 3D Plotly map saved to {save_path}")
        return fig

    def create_subway_network_graph(self, save_path='subway_network.html'):
        """
        Create an interactive network graph of subway connections
        """
        # Build network of stations
        stations_by_line = {}

        for idx, station in self.metro.iterrows():
            lines = str(station.get('Daytime Routes', '')).strip().split()
            for line in lines:
                if line not in stations_by_line:
                    stations_by_line[line] = []
                stations_by_line[line].append({
                    'name': station.get('Stop Name', ''),
                    'lon': station.geometry.x,
                    'lat': station.geometry.y
                })

        fig = go.Figure()

        # Add each line as a separate trace
        for line, stations in stations_by_line.items():
            if len(stations) < 2:
                continue

            lons = [s['lon'] for s in stations]
            lats = [s['lat'] for s in stations]
            names = [s['name'] for s in stations]

            color = self.subway_colors.get(line, '#808183')

            # Add line connections
            fig.add_trace(go.Scattermapbox(
                lon=lons,
                lat=lats,
                mode='markers+lines',
                marker=dict(size=10, color=color),
                line=dict(width=3, color=color),
                text=[f"{line} Train - {name}" for name in names],
                hovertemplate='<b>%{text}</b><extra></extra>',
                name=f'{line} Line'
            ))

        # Update layout
        fig.update_layout(
            title='🚇 NYC Subway Network Map',
            mapbox=dict(
                style='carto-darkmatter',
                center=dict(lat=40.7128, lon=-74.0060),
                zoom=10
            ),
            height=800,
            showlegend=True
        )

        fig.write_html(save_path)
        print(f"✨ Subway network graph saved to {save_path}")
        return fig

    def create_density_heatmap(self, save_path='density_heatmap.html'):
        """
        Create density heatmap using Plotly
        """
        fig = go.Figure()

        # Metro density
        metro_lons = [s.geometry.x for _, s in self.metro.iterrows()]
        metro_lats = [s.geometry.y for _, s in self.metro.iterrows()]

        fig.add_trace(go.Densitymapbox(
            lon=metro_lons,
            lat=metro_lats,
            radius=20,
            colorscale='Hot',
            name='Metro Density',
            showscale=True
        ))

        fig.update_layout(
            title='🔥 NYC Metro Station Density Heatmap',
            mapbox=dict(
                style='carto-darkmatter',
                center=dict(lat=40.7128, lon=-74.0060),
                zoom=10
            ),
            height=700
        )

        fig.write_html(save_path)
        print(f"✨ Density heatmap saved to {save_path}")
        return fig


# Example usage
if __name__ == "__main__":
    # Load data
    parks = pd.read_csv("assets/parks.csv", low_memory=False)
    metro = pd.read_csv("assets/metro.csv", low_memory=False)
    rest = pd.read_csv("assets/restaurants.csv", low_memory=False)

    # Create visualizer
    viz = NYCMapVisualizer(parks, metro, rest)

    # Create all the awesome maps
    print("🎨 Creating epic visualizations...")

    # 1. Folium interactive map
    folium_map = viz.create_folium_masterpiece(save_path='nyc_epic_map.html')

    # 2. 3D Plotly visualization
    plotly_3d = viz.create_plotly_3d_masterpiece(save_path='nyc_3d_view.html')

    # 3. Subway network graph
    subway_network = viz.create_subway_network_graph(save_path='subway_network.html')

    # 4. Density heatmap
    density_map = viz.create_density_heatmap(save_path='density_heatmap.html')

    print("\n🎉 All visualizations created successfully!")
    print("📂 Open the HTML files in your browser to see the magic! ✨")

FileNotFoundError: [Errno 2] No such file or directory: 'sample_data/parks.csv'